# Get started

Node Classification

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

DGL(Deep Graph Library)은 backend로 pytorch tensorflow mxnet을 사용할 수 있다.([참조](https://docs.dgl.ai/en/0.4.x/install/backend.html) )

In [3]:
!pip install dgl
import dgl
from dgl import DGLGraph

dgl.load_backend('pytorch')

Using backend: pytorch
Using backend: pytorch


# GNN 모델

Node representation.
DGL의 사용방법 2가지:
* nn.Module
* message passing방법

GraphSage : 
$$h_{N(v)}^{(l)} \gets AGGREGATE_k({h_u^{(l-1)}, \forall u \in N(v)})$$
$$h_v^{(l)} \gets \sigma(W^k \cdot CONCAT(h_v^{(l-1)}, h_{N(v)}^{(l)})),$$

where $N(v)$ is the neighborhood of node $v$ and $l$ is the layer Id.
$N(v)$ 는 노드 $v$의 이웃. $l$는 레이어 번호(Id).

GraphSage는 multi-Layer로 구성된다. 각 레이어에서 노드는 edge로 이어진 이웃 노드에 접근한다. 레이어가 k개이면, k와 k보다 작은 수 만큼 떨어진 node와의 관계를 본다. k만큼 멀리있는 노드를 k-hop이라고 한다. GraphSage의 output은 각 node의 representation이다.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/GNN.png" alt="drawing" width="600"/>

여기에선, DGL의 `nn` 모듈을 사용한다. `SAGEConv`가 `GraphSage`의 각 레이어에 들어가있다.

In [4]:
from dgl.nn.pytorch import conv as dgl_conv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(dgl_conv.SAGEConv(in_feats, n_hidden, aggregator_type,
                                         feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(dgl_conv.SAGEConv(n_hidden, n_hidden, aggregator_type,
                                             feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(dgl_conv.SAGEConv(n_hidden, out_dim, aggregator_type,
                                         feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h)
        return h

다양한 `GraphConv`의 예제.([참조](https://docs.dgl.ai/en/0.4.x/tutorials/models/))

# 데이터셋

DGL안에 데이터셋이 많다. [참조](https://doc.dgl.ai/api/python/data.html#dataset-classes)

여기에선 Pubmed라는 논문인용 네트워크 데이터넷을 사용한다.

노드는 각 paper이고, edge는 인용된 두 논문을 이어준다. 19,717 논문(node)들과 88,651 edge가 있다. 각 node는 feature와 class를 가지고 있다(sparse bag-of-words).

In [5]:
from dgl.data import citegrh

data = citegrh.load_pubmed()

features = torch.FloatTensor(data.features)     # feature를 넘파이에서 파이토치 텐서로 변환
in_feats = features.shape[1]                    # dimension = 500
labels = torch.LongTensor(data.labels)          # 파이토치 텐서로 변환
n_classes = data.num_labels                     # 클래스 갯수

Finished data loading and preprocessing.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000


네트워크(graph) 구조를 [NetworkX](https://networkx.github.io) 객체를 쓴다.

In [6]:
import networkx as nx
data.graph.remove_edges_from(nx.selfloop_edges(data.graph)) #self-loop제거

In [7]:
g = DGLGraph(data.graph)    #DGL그래프로 변환
g.readonly()                #오로지 더 빠르게 읽기 위해. 수정이 없는경우 사용. (메모리 절약)

# Semi-supervised node classification

모든 graph를 가지고 있으며, 각 node의 feature를 가지고 있고, 몇개만 class label이 있으며, 몇개는 없다. class label이 없는 node의 class를 예측한다. labeled node와 unlabeled node를 같이 학습한다.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/node_classify1.png" alt="drawing" width="200"/>


2층 GraphSage

In [8]:
n_hidden = 64
n_layers = 2
dropout = 0.5
aggregator_type = 'gcn'

gconv_model = GraphSAGEModel(in_feats,
                             n_hidden,
                             n_classes,
                             n_layers,
                             F.relu,
                             dropout,
                             aggregator_type)

`GraphSage`는 DGLGraph객체와 node의 feature들을 input으로 받는다. output은 node embeddings이다. node embeddings로 cross entropy loss를 써서 훈련한다.



In [9]:
class NodeClassification(nn.Module):
    def __init__(self, gconv_model, n_hidden, n_classes):
        super(NodeClassification, self).__init__()
        self.gconv_model = gconv_model
        self.loss_fcn = torch.nn.CrossEntropyLoss()

    def forward(self, g, features, train_mask):
        logits = self.gconv_model(g, features)
        return self.loss_fcn(logits[train_mask], labels[train_mask])

Test node에서 classification정확도 함수

In [10]:
def NCEvaluate(model, g, features, labels, test_mask):
    model.eval()
    with torch.no_grad():
        logits = model.gconv_model(g, features)
        logits = logits[test_mask]                          #Test set에서만 accuracy 계산
        test_labels = labels[test_mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == test_labels)
        return correct.item() * 1.0 / len(test_labels)

In [17]:
# train/val/text mask 생성
train_mask = torch.BoolTensor(data.train_mask)
val_mask = torch.BoolTensor(data.val_mask)
test_mask = torch.BoolTensor(data.test_mask)

print("""----Data statistics------'
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_classes,
           data.train_mask.sum().item(),
           data.val_mask.sum().item(),
           data.test_mask.sum().item()))

----Data statistics------'
      #Classes 3
      #Train samples 60
      #Val samples 500
      #Test samples 1000


훈련

In [18]:
model = NodeClassification(gconv_model, n_hidden, n_classes)

# 하이퍼파라미터
weight_decay = 5e-4
n_epochs = 150
lr = 1e-3

# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

dur = []
for epoch in range(n_epochs):
    model.train()
    
    loss = model(g, features, train_mask)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = NCEvaluate(model, g, features, labels, val_mask)
    print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f}"
          .format(epoch, loss.item(), acc))

acc = NCEvaluate(model, g, features, labels, test_mask)
print("\nTest Accuracy {:.4f}".format(acc))

Epoch 00000 | Loss 1.0792 | Accuracy 0.3880
Epoch 00001 | Loss 1.0760 | Accuracy 0.3880
Epoch 00002 | Loss 1.0748 | Accuracy 0.3880
Epoch 00003 | Loss 1.0666 | Accuracy 0.3880
Epoch 00004 | Loss 1.0647 | Accuracy 0.3880
Epoch 00005 | Loss 1.0642 | Accuracy 0.3880
Epoch 00006 | Loss 1.0599 | Accuracy 0.3880
Epoch 00007 | Loss 1.0569 | Accuracy 0.3880
Epoch 00008 | Loss 1.0492 | Accuracy 0.3880
Epoch 00009 | Loss 1.0500 | Accuracy 0.3880
Epoch 00010 | Loss 1.0488 | Accuracy 0.3880
Epoch 00011 | Loss 1.0418 | Accuracy 0.3880
Epoch 00012 | Loss 1.0437 | Accuracy 0.3880
Epoch 00013 | Loss 1.0370 | Accuracy 0.3920
Epoch 00014 | Loss 1.0388 | Accuracy 0.4040
Epoch 00015 | Loss 1.0383 | Accuracy 0.4220
Epoch 00016 | Loss 1.0359 | Accuracy 0.4400
Epoch 00017 | Loss 1.0313 | Accuracy 0.4500
Epoch 00018 | Loss 1.0329 | Accuracy 0.4600
Epoch 00019 | Loss 1.0251 | Accuracy 0.4760
Epoch 00020 | Loss 1.0269 | Accuracy 0.4940
Epoch 00021 | Loss 1.0240 | Accuracy 0.5040
Epoch 00022 | Loss 1.0206 | Accu